<h1 style='color: green; font-size: 30px; font-weight: bold;'>Data Science - Regressão Logística: de baixo dos panos</h1>

# <font color='black' style='font-size: 24px;'>1.1 Conhecendo o Dataset</font>
<hr style='border: 2px solid black;'>

## Importando o numpy, pandas e random

In [1]:
import numpy as np
import pandas as pd
import random
np.random.seed(42)

## O Dataset e o Projeto
<hr>

### Descrição:
<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>O mercado imobiliário vem sendo objeto de diversos estudos e pesquisas nos últimos tempos. A crise financeira que afeta a economia tem afetado significativamente os investimentos e ganhos advindos deste setor. Este cenário incentiva o aumento do interesse por estudos de previsão de demanda baseados em características deste mercado, dos imóveis e do entorno destes imóveis.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Neste contexto o objetivo principal do nosso projeto é desenvolver um sistema de avaliação imobiliária utilizando a metodologia de regressões lineares que é uma das técnicas de machine learning.</p>

<p style='font-size: 15px; line-height: 2; margin: 10px 50px; text-align: justify; text-indent: 35px;'>Nosso dataset é uma amostra aleatória de tamanho 5000 de imóveis disponíveis para venda no município do Rio de Janeiro.</p>

### Dados:
<ul style='font-size: 15px; line-height: 2; text-align: justify;'>
    <li><b>Valor</b> - Valor (R$) de oferta do imóvel;</li>
    <li><b>Area</b> - Área do imóvel em m².</li>
    <li><b>Dist_Praia</b> - Distância do imóvel até a praia (km) (em linha reta).</li>
    <li><b>Dist_Farmacia</b> - Distância do imóvel até a farmácia mais próxima (km) (em linha reta).</li>
    <li><b>Vale_a_pena_comprar</b> - Valor booleano indicando se vale a pena comprar este imóvel.</li>
</ul>

In [2]:
dados = pd.read_csv('dados_classificacao_multivariavel.csv')

In [3]:
dados.head()

,Valor,Area,Dist_Praia,Dist_Farmacia,vale_a_pena_comprar
0,4600000,280,0.240925,0.793637,1
1,900000,208,0.904136,0.134494,1
2,2550000,170,0.059525,0.423318,1
3,550000,100,2.883181,0.525064,0
4,2200000,164,0.239758,0.192374,1


In [4]:
valor = np.log(dados['Valor'])
area = np.log(dados['Area'])
dist_Praia = np.log(dados['Dist_Praia'] + 1)
dist_Farmacia = np.log(dados['Dist_Farmacia'] + 1)
vale_a_pena_comprar = dados['vale_a_pena_comprar']


valor = np.array(valor)
area = np.array(area)
dist_Praia = np.array(dist_Praia)
dist_Farmacia = np.array(dist_Farmacia)
vale_a_pena_comprar = np.array(vale_a_pena_comprar)


X = np.array([valor, area, dist_Praia, dist_Farmacia]).T
y = vale_a_pena_comprar

# Regressão logística multivariável

Agora nosso problema se trata de uma classificação. Quero saber se vale a pena comprar um determinado imóvel. Para isto, vamos usar uma curva logística no lugar de uma linha. A função logística é uma função que varia entre 0 e 1, e é contínuo em todo seu domínio, definida da seguinte forma:

$$
y = \frac{1}{1+e^{-x}}
$$

Podemos definir isto a partir da notação matricial:

$$
y = \frac{1}{1+e^{-X\bullet \theta}}
$$

Onde $\theta$ são os parâmetros que devem ser otimizados e X são os atributos da amostra em questão.

Desta forma, nosso modelo irá retornar a probabilidade de nossa amostra ser da classe em questão.

In [5]:
def prever_prob(theta, X):
    z = np.dot(X, theta)
    return 1 / (1 + np.exp(-z))

E para classificar nossaas amostras, devemos simplesmente checar se a probabilidade está acima de um limiar desejado (geralmente 0.5).

In [6]:
def classificar(y, limiar):
    return y > limiar

Agora, vamos a função de custo. Usaremos a função da entropia cruzada.

$$
Entropia Cruzada = \frac{1}{N}\sum_{i=1}^N(-y_ilog(y^p_i) - (1 - y_i)log(1 - y^p_i))
$$

In [7]:
def entropia_cruzada(previsto, y):
    return (-y * np.log(previsto) - (1 - y) * np.log(1 - previsto)).mean()

Devemos minimizar nossa função de custo, para isto usamos o gradiente descendente. Nele, usamos a derivada de nossa função de custo, definida como:

$$
EntropiaCruzada' = \frac{1}{N}X^T\bullet(y^p - y)
$$

In [8]:
def gradienteDescendente(theta, X, y, alpha):
    previsto = prever_prob(theta, X)
    erro = previsto - y
    
    gradiente = np.dot(X.T, erro) / (len(X))

    theta -= alpha*gradiente

O método c_ do numpy vai apenas colocar mais um atributo em todas amostras de nosso conjunto. Este ultimo atributo será sempre unitário, e será responsável pela variável independente $\theta_0$.

Após isto, vamos criar nossos pesos de forma aleatória e executar a nossa otimização.

In [9]:
X_ = X

X = np.c_[np.ones(X.shape[0]), X] 

theta = np.random.rand(X.shape[1])

for i in range(7000):
    previsto = prever_prob(theta, X)
    custo = entropia_cruzada(previsto, y)
    
    if i % 1000 == 0:
        print(custo)
    
    gradienteDescendente(theta, X, y, 0.1)

8.64039360110891
0.3068509524080488
0.2988254953813712
0.29835810745636665
0.2980646560845412
0.2978085504282278
0.2975664466572867


In [10]:
theta

array([ 0.9962117 , -0.64682391,  2.49355438, -3.63137227,  0.11369866])

In [11]:
classificados = classificar(prever_prob(theta, X), 0.5)

Para checar a eficiencia do modelo, vamos usar a acurácia.

In [12]:
(classificados == y).mean()

0.8934

# Comparando com o Sk-learn

In [13]:
from sklearn.linear_model import LogisticRegression

In [14]:
lr = LogisticRegression(random_state=42, solver='lbfgs')

In [15]:
lr.fit(X_, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
(lr.predict(X_) == y).mean()

0.8904

# Divisão conjuntos treinamento e teste

É justo avaliar nosso modelo com os mesmos dados que ele foi treinado?

Para melhor avaliar nosso modelo, devemos dividir o conjunto de dados em dois. Um conjunto para o treinamento e outro para o teste.

In [17]:
def divisao_treinamento_teste(X, y, porcentagem_teste, random_seed = 42):
    random.seed(random_seed)
    
    X_test, y_test = [], []
    
    X_train = list(X)
    y_train = list(y)
    
    tam_y = porcentagem_teste * len(y)
    
    while len(y_test) < tam_y:
        index = random.randrange(len(X_train))
        X_test.append(X_train.pop(index))
        y_test.append(y_train.pop(index))
        
    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [18]:
X_train, X_test, y_train, y_test = divisao_treinamento_teste(X_, y, 0.2)

Agora sim! Temos um conjunto de dados para o treinamento e outro para o teste. Então vamos treinar nosso modelo usando o conjunto de treinamento.

In [19]:
X_train = np.c_[np.ones(X_train.shape[0]), X_train] 

theta = np.random.rand(X_train.shape[1])

for i in range(7000):
    previsto = prever_prob(theta, X_train)
    custo = entropia_cruzada(previsto, y_train)
    
    if i % 1000 == 0:
        print(custo)
    
    gradienteDescendente(theta, X_train, y_train, 0.1)

3.028514164989356
0.3179232424610933
0.30839164306544037
0.30571640814580736
0.30514379888268917
0.30486037452781517
0.3045890579056707


In [20]:
classificados = classificar(prever_prob(theta, X_train), 0.5)
print((classificados == y_train).mean())

0.891


Para validar nosso modelo, vamos apenas calcular sua acurácia usando o conjunto de teste.

In [21]:
X_test = np.c_[np.ones(X_test.shape[0]), X_test] 
classificados = classificar(prever_prob(theta, X_test), 0.5)
print((classificados == y_test).mean())

0.9
